
# **[군장병 AI 기본1] Image Processing**

* ### Hak Gu Kim, Ph.D.
  * ### Assistant Professor
  * ### Graduate School of Advanced Imaging Science, Multimedia & Film (GSAIM)
  * ### Chung-Ang University
  * ### Webpage: www.irislab.cau.ac.kr


# **Programming Practice III: Panorama Stitching**

### 1. Understande SIFT descriptor
### 2. Implement RANSAC algorithm with SIFT descriptor features
### 3. Perform your own panorama images using two view images

## **[Step 0]** Environmental Setting

In [1]:
!pip install opencv-contrib-python==4.10.0.84

In [2]:
# Connect to the google drive #

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Import the required libraries for image processing

import sys
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import math
import random
from tqdm.notebook import tqdm
plt.rcParams['figure.figsize'] = [15, 15]

# Define the directory
dir = '/content/gdrive/My Drive/Colab Notebooks/military_ai3/' # File path

In [4]:
# Define the functions for the load and save the input image

def loadImg(in_fname):
  img = cv2.imread(dir + in_fname)

  if img is None:
    print('Image load failed!')
    sys.exit()

  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.title("Input RGB Image")
  plt.show()

  return img

## Save image file
def saveImg(out_img, out_fname):
  cv2.imwrite(dir + out_fname, out_img)

In [ ]:
# Homography를 계산하는 함수 정의
##############################################
#          Transformation Matrix, H          #
##############################################
def homography(pairs):
    rows = []
    for i in range(pairs.shape[0]):
        # 각 대응점 쌍에 대해 변환을 위한 행렬 요소를 계산
        p1 = np.append(pairs[i][0:2], 1)  # 첫 번째 이미지의 좌표 (x, y)
        p2 = np.append(pairs[i][2:4], 1)  # 두 번째 이미지의 좌표 (x', y')

        # 변환 행렬을 구하기 위한 A 행렬의 두 개의 행(row)을 정의
        row1 = [0, 0, 0, p1[0], p1[1], p1[2], -p2[1]*p1[0], -p2[1]*p1[1], -p2[1]*p1[2]]
        row2 = [p1[0], p1[1], p1[2], 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1], -p2[0]*p1[2]]

        # 두 개의 행(row)을 리스트에 추가
        rows.append(row1)
        rows.append(row2)

    # numpy 배열로 변환
    rows = np.array(rows)

    # SVD(특이값 분해)를 통해 행렬의 최소 자승 해를 구함
    U, s, V = np.linalg.svd(rows)

    # 마지막 특이 벡터가 변환 행렬 H가 됨
    H = V[-1].reshape(3, 3)

    # 표준화하여 마지막 요소를 1로 설정
    H = H/H[2, 2]
    return H


# 파노라마 이미지 생성 함수 정의
##############################################
#         Panoramic Image Generation         #
##############################################
def stitchImg(left, right, H):
    print("stitching image ...")

    # 이미지를 double 타입으로 변환하고 노이즈를 줄이기 위해 정규화
    left = cv2.normalize(left.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)
    right = cv2.normalize(right.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # 왼쪽 이미지의 코너 좌표를 계산하고 변환 행렬 H를 적용하여 새로운 좌표를 계산
    height_l, width_l, channel_l = left.shape
    corners = [[0, 0, 1], [width_l, 0, 1], [width_l, height_l, 1], [0, height_l, 1]]
    corners_new = [np.dot(H, corner) for corner in corners]
    corners_new = np.array(corners_new).T
    x_news = corners_new[0] / corners_new[2]
    y_news = corners_new[1] / corners_new[2]
    y_min = min(y_news)
    x_min = min(x_news)

    # 새로운 이미지의 좌표를 보정하기 위한 변환 행렬을 정의
    translation_mat = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])
    H = np.dot(translation_mat, H)

    # 새로운 이미지의 크기를 계산
    height_new = int(round(abs(y_min) + height_l))
    width_new = int(round(abs(x_min) + width_l))
    size = (width_new, height_new)

    # 왼쪽 이미지를 변환 행렬 H를 이용해 왜곡(warping)시킴
    warped_l = cv2.warpPerspective(src=left, M=H, dsize=size)

    height_r, width_r, channel_r = right.shape
    height_new = int(round(abs(y_min) + height_r))
    width_new = int(round(abs(x_min) + width_r))
    size = (width_new, height_new)

    # 오른쪽 이미지를 보정된 좌표로 왜곡(warping)시킴
    warped_r = cv2.warpPerspective(src=right, M=translation_mat, dsize=size)

    black = np.zeros(3)  # 검정색 픽셀 (빈 영역)

    # 이미지 결합 절차, 결과는 warped_l에 저장됨
    for i in tqdm(range(warped_r.shape[0])):
        for j in range(warped_r.shape[1]):
            pixel_l = warped_l[i, j, :]
            pixel_r = warped_r[i, j, :]

            # 왼쪽 픽셀과 오른쪽 픽셀을 비교하여 하나라도 유효한 경우 결합
            if not np.array_equal(pixel_l, black) and np.array_equal(pixel_r, black):
                warped_l[i, j, :] = pixel_l
            elif np.array_equal(pixel_l, black) and not np.array_equal(pixel_r, black):
                warped_l[i, j, :] = pixel_r
            elif not np.array_equal(pixel_l, black) and not np.array_equal(pixel_r, black):
                warped_l[i, j, :] = (pixel_l + pixel_r) / 2
            else:
                pass

    # 최종적으로 두 이미지를 결합한 파노라마 이미지 반환
    stitch_image = warped_l[:warped_r.shape[0], :warped_r.shape[1], :]
    return stitch_image


# SIFT 특징을 그리는 함수 정의
def plot_sift(gray_img, rgb_img, keypnt):
    tmp_img = rgb_img.copy()
    sift_on_img = cv2.drawKeypoints(gray_img, keypnt, tmp_img, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return sift_on_img


# 매칭된 특징점을 그리는 함수 정의
def plot_matches(matches, total_img):
    match_img = total_img.copy()
    offset = total_img.shape[1]/2
    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(np.array(match_img).astype('uint8')) # RGB는 정수형 타입이어야 함

    ax.plot(matches[:, 0], matches[:, 1], 'xr')
    ax.plot(matches[:, 2] + offset, matches[:, 3], 'xr')

    ax.plot([matches[:, 0], matches[:, 2] + offset], [matches[:, 1], matches[:, 3]], 'r', linewidth=0.5)

    plt.show()


## **[Practice III-1]** Understande SIFT descriptor feature and matching
* Following codes are SIFT Feature Extraction and Matching
* There is nothing for you to do in this field

In [5]:
##############################################
#             Feature Extraction             #
##############################################
def SIFT(img):
    # SIFT 특징점 검출기를 생성합니다.
    # OpenCV 버전에 따라 xfeatures2d 모듈을 사용할 수도 있고, 기본 SIFT를 사용할 수도 있습니다.
    siftDetector= cv2.xfeatures2d.SIFT_create()  # 최대 1000개의 포인트로 제한

    # 이미지에서 SIFT 특징점을 검출하고, 해당 특징점의 기술자(descriptor)를 계산합니다.
    kp, des = siftDetector.detectAndCompute(img, None)

    # 검출된 특징점(keypoints)과 그에 대응하는 기술자(descriptors)를 반환합니다.
    return kp, des


##############################################
#               Feature Matching             #
##############################################
def siftMatch(kp1, des1, img1, kp2, des2, img2, threshold):
    # Brute-Force 매처를 생성합니다.
    bf = cv2.BFMatcher()

    # KNN 알고리즘을 사용하여 가장 가까운 두 개의 매칭을 찾습니다.
    matches = bf.knnMatch(des1, des2, k=2)

    # 비율 테스트(Ratio Test)를 적용하여 좋은 매칭을 걸러냅니다.
    good = []
    for m, n in matches:
        if m.distance < threshold * n.distance:
            good.append([m])

    # 좋은 매칭을 기반으로 매칭된 특징점 좌표를 수집합니다.
    matches = []
    for pair in good:
        # 첫 번째 이미지의 특징점 좌표와 두 번째 이미지의 특징점 좌표를 결합하여 저장
        matches.append(list(kp1[pair[0].queryIdx].pt + kp2[pair[0].trainIdx].pt))

    # 매칭된 특징점을 numpy 배열로 변환하여 반환합니다.
    matches = np.array(matches)
    return matches


## ## **[Practice III-2]** Implement RANSAC algorithm with SIFT descriptor features
* This is main part in this practice
* Carefully read and understand the following RANSAC code,
* Then please **complete the incomplete code:** `varable_name = [ ]`

In [ ]:
##############################################
#                   RANSAC                   #
##############################################
def ransac(matches, k_samples, threshold, iters):
    num_best_inliers = 0  # 현재까지 찾은 최고의 인라이어 수를 저장

    for i in range(iters):
        # 무작위로 k개의 매칭된 점을 선택하여 랜덤 샘플링
        rand_pnt = randomPoint(matches, k_samples)
        # 선택된 점들로부터 호모그래피 행렬 H 계산
        H = homography(rand_pnt)

        # 계산된 H가 유효하지 않은 경우 건너뜀 (H의 순위가 3 미만일 경우)
        if np.linalg.matrix_rank(H) < 3:
            continue

        # 모든 매칭된 점들에 대해 오차 계산
        errs = getError(matches, H)
        # 오차가 임계값(threshold)보다 작은 점들을 인라이어로 선택
        idx = np.where(errs < threshold)[0]
        inliers = matches[idx]

        num_inliers = len(inliers)  # 현재 반복에서의 인라이어 수
        # 인라이어 수가 지금까지 최적의 모델보다 많다면 업데이트
        if num_inliers > num_best_inliers:
            ############################################
            ##       COMPLETE THE FOLLOWING CODE      ##
            ############################################
            # 최고의 인라이어와 호모그래피 행렬을 저장
            best_inliers = inliers.copy()
            num_best_inliers = num_inliers
            best_H = H.copy()

    # 최종적으로 최고의 인라이어 수와 호모그래피 행렬 출력
    print("inliers/matches: {}/{}".format(num_best_inliers, len(matches)))
    return best_inliers, best_H  # 최적의 인라이어와 호모그래피 행렬 반환


##############################################
#        랜덤으로 매칭된 점을 선택하는 함수   #
##############################################
def randomPoint(matches, k_samples):
  ############################################
  ##       COMPLETE THE FOLLOWING CODE      ##
  ############################################
  # k_samples: 무작위로 선택할 점의 수
  # 매칭된 점들 중 무작위로 k_samples 개의 점을 선택
  idx = random.sample(range(len(matches)), k_samples)
  rand_pnt = [matches[i] for i in idx]

  return np.array(rand_pnt)  # 선택된 점들을 numpy 배열로 반환


##############################################
#     호모그래피를 사용해 오차를 계산하는 함수  #
##############################################
def getError(rand_pnt, H):
    num_points = len(rand_pnt)  # 주어진 점들의 수
    # 첫 번째 이미지의 점들에 1을 추가하여 3차원 좌표로 변환
    all_p1 = np.concatenate((rand_pnt[:, 0:2], np.ones((num_points, 1))), axis=1)
    # 두 번째 이미지의 실제 점들 좌표
    all_p2 = rand_pnt[:, 2:4]

    est_p2 = np.zeros((num_points, 2))  # 예측된 두 번째 이미지의 좌표 초기화
    for i in range(num_points):
        ############################################
        ##       COMPLETE THE FOLLOWING CODE      ##
        ############################################
        # 호모그래피 행렬을 사용해 첫 번째 이미지의 점들을 변환
        temp_est_p2 = np.dot(H, all_p1[i])

        # 변환된 좌표를 2D로 정상화하여 예측된 p2 좌표를 얻음
        est_p2[i] = (temp_est_p2 / temp_est_p2[2])[0:2]

    ############################################
    ##       COMPLETE THE FOLLOWING CODE      ##
    ############################################
    # 실제 좌표(all_p2)와 예측된 좌표(est_p2) 간의 거리(오차)를 계산
    errors = np.linalg.norm(all_p2 - est_p2, axis=1) ** 2

    return errors  # 계산된 오차 반환


## **[Practice III-3]** Perform your own panorama images using two view images

* Try to change various variables such as the number of samples, threshold, the number of iterations in RANSAC function
* Try to get your own panorama image with the pictures you took

In [ ]:
##############################################
#                Main Function               #
##############################################

if __name__ == "__main__":

    # 파일 이름 설정
    fname_left = 'img_left.jpg'
    fname_right = 'img_right.jpg'

    # 이미지를 읽고 크기를 720x720으로 조정
    img_left  = cv2.resize(cv2.imread(dir + fname_left), dsize=(720, 720))
    img_right = cv2.resize(cv2.imread(dir + fname_right), dsize=(720, 720))

    # 이미지를 그레이스케일로 변환
    gray_left = cv2.cvtColor(img_left, cv2.COLOR_RGB2GRAY)
    gray_right = cv2.cvtColor(img_right, cv2.COLOR_RGB2GRAY)

    # SIFT를 사용해 특징점과 디스크립터를 추출
    kp_left, des_left = SIFT(gray_left)
    kp_right, des_right = SIFT(gray_right)

    # 추출된 특징점의 수와 디스크립터의 크기를 출력
    print(len(kp_left))   # 왼쪽 이미지에서 추출된 특징점의 수
    print(des_left.shape) # 각 특징점에 대한 128차원의 디스크립터

    # 특징점을 시각화하여 이미지를 생성
    kp_left_img = plot_sift(gray_left, img_left, kp_left)
    kp_right_img = plot_sift(gray_right, img_right, kp_right)

    # SIFT를 통해 매칭된 특징점들 계산
    matches = siftMatch(kp_left, des_left, img_left, kp_right, des_right, img_right, 0.5)

    # 왼쪽과 오른쪽 이미지를 가로로 결합하여 매칭된 결과를 시각화
    temp_img = np.concatenate((img_left, img_right), axis=1)
    plot_matches(matches, temp_img) # 매칭된 특징점 시각화

    # 매칭된 특징점의 수를 출력
    print(matches.shape)  # 잘 매칭된 특징점(대응점)의 수

    # RANSAC 알고리즘을 사용하여 아웃라이어를 제거하고 인라이어와 호모그래피 행렬을 찾음
    inliers, H = ransac(matches, 400, 0.5, 1000)

    # 인라이어를 기반으로 매칭된 결과를 시각화
    plot_matches(inliers, temp_img) # 인라이어 매칭 시각화

    # 최종적으로 파노라마 이미지를 생성하고 출력
    panoramic_img = stitchImg(img_left, img_right, H) * 255
    cv2_imshow(panoramic_img)


Output hidden; open in https://colab.research.google.com to view.